In [3]:
import pandas as pd
from pathlib import Path
import sys, os
import datetime
import pandas as pd
import time

from ut_msba_edgar_scraper import Downloader

In [4]:
downloader = Downloader()

In [5]:
fallen_angels = pd.read_csv('fallen_angels.csv')
fallen_angels.datadate = pd.to_datetime(fallen_angels.datadate)
fallen_angels

,datadate,gvkey,tic
0,1993-04-30,7063,HST
1,2001-05-31,8692,PCH
2,2008-09-30,15142,STAR
3,2008-09-30,27780,SITC
4,2008-09-30,30384,FR
5,2014-03-31,30637,VRE
6,2006-11-30,174022,RLGY


In [6]:
# fallen_angels.iloc[0].datadate.strftime('%Y-%m-%d')
date = fallen_angels.iloc[0].datadate
date.strftime('%Y-%m-%d'), date.replace(year = date.year -2).strftime('%Y-%m-%d')


('1993-04-30', '1991-04-30')

In [7]:
def fetch_reports(args):
    downloader = Downloader()
    report_type, date, gvkey = args
    date = datetime.datetime(year = 2002, month=1, day= 1) if date.year <2000 else date
    after_date = date.replace(year = date.year -2) if date.year >=2002 else datetime.datetime(year = 2000, month=1, day= 1)
    
    print(f"{report_type}, {date.strftime('%Y-%m-%d')}, {after_date.strftime('%Y-%m-%d')}, {gvkey}")
    
    filings_list = downloader.get_filings(report_type,
                                        gvkey, 
                                        before= date.strftime('%Y-%m-%d'),
                                        after = after_date.strftime('%Y-%m-%d'))
    
    for filing in filings_list:
        with open(f'data/{repr(filing)}.html','wb') as f:
            f.write(filing.get_report())
    

In [8]:
os.makedirs('data', exist_ok = True)
for index, rows in fallen_angels.iterrows():
    if rows.datadate.year >= 2000:
        for report_type in ['10-K','10-Q']:
            fetch_reports((report_type, rows.datadate,rows.gvkey))


10-K, 2001-05-31, 2000-01-01, 8692
10-Q, 2001-05-31, 2000-01-01, 8692
10-K, 2008-09-30, 2006-09-30, 15142
10-Q, 2008-09-30, 2006-09-30, 15142
10-K, 2008-09-30, 2006-09-30, 27780
10-Q, 2008-09-30, 2006-09-30, 27780
10-K, 2008-09-30, 2006-09-30, 30384
10-Q, 2008-09-30, 2006-09-30, 30384
10-K, 2014-03-31, 2012-03-31, 30637
10-Q, 2014-03-31, 2012-03-31, 30637
10-K, 2006-11-30, 2004-11-30, 174022
10-Q, 2006-11-30, 2004-11-30, 174022
